In [ ]:
'''
Andy Liu, Justin Jiang - 7/19/20
Math189R Final Project
Livesent_Dataprocessing

Input - articles, doc-topics from topic modeling.
Output - spreadsheet containing live sentiment for each minute of news coverage.
'''

In [1]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
articles = pd.read_csv('./articles.csv')
doctopics = pd.read_csv('./doc-topics (3).csv')

In [62]:
articles.head()

,Unnamed: 0,title,datetime,text,url
0,0,"As Trump refuses to lead, America tries to sav...",2020-07-16 01:24:09.385063,(CNN) President Donald Trump isn't leading Ame...,https://www.cnn.com/2020/07/16/politics/donald...
1,1,Fauci holds his ground against Navarro broadside,2020-07-16 01:24:10.364113,"Fauci, in his most forceful pushback to date a...",https://www.cnn.com/2020/07/15/politics/donald...
2,2,Nick Cannon Finally Offers Apology for Anti-Se...,2020-07-16 01:24:12.387678,"3 . This CBD Gel Is Great for My Aches, Pains,...",https://www.thedailybeast.com/nick-cannon-offe...
3,3,Blinded by science: A flailing president attac...,2020-07-16 01:24:21.849006,"Fauci, who sounded the alarm about the serious...",https://www.nydailynews.com/opinion/ny-edit-fa...
4,4,Brad Parscale out as President Trump's campaig...,2020-07-16 01:24:24.728675,\n,https://www.youtube.com/watch?v=X0I_uxGpSXE


In [50]:
def getnumber(s):
    s1 = s.split(':')[-1]
    n = (int(s1))
    return(n)

In [54]:
#convert docname column to indices, group topic/proportion data by index
doctopics['docname'] = doctopics['docname'].apply(lambda x: getnumber(x))
dt = doctopics.groupby('docname', as_index = False).agg(lambda x: list(x))
dt.head()

,docname,topic,proportion
0,31,0,0.553972
1,31,1,0.446028
2,66,23,0.593337
3,66,54,0.136041
4,66,15,0.128014


In [58]:
analyzer = SentimentIntensityAnalyzer()

In [68]:
numTopics = 100
#below functions used to get topic sentiment analysis data per article
def getText(docname):
    '''
    input - docname from a row of dt
    output - text from that row of the other pd
    '''
    text = (articles.iloc[docname])['text']
    return(text)
    
def weights(docname, topics, proportions):
    '''
    input - docname, topic, proportion from a row
    output - a dictionary containing all 100 topics as keys and sentiments as values
    '''
    output_dict = {}
    sentiment = analyzer.polarity_scores(getText(docname))
    for i in topics:
        output_dict[i] = sentiment['compound']*proportions[topics.index(i)]
    for n in range(0,numTopics):
        if not n in output_dict:
            output_dict[n] = 0
    return(output_dict)

In [69]:
output_d = {}
dates = []
docname_l = list(dt.docname)
topic_l = list(dt.topic)
proportion_l = list(dt.proportion)

In [1]:
#iterate through dataframe rows, calculate text sentiments of each article, multiply by weights of each topic
for i in range(len(dt.topic)):
    try:
        dates.append((articles.iloc[docname_l[i]])['datetime'])
        w = weights(docname_l[i], topic_l[i], proportion_l[i])
        for j in range(numTopics):
            try:
                output_d[j].append(w[j])
            except KeyError:
                output_d[j] = [w[j]]
                
    except Exception as e:
        pass

NameError: name 'dt' is not defined

In [72]:
#add outputs to dataframe
dt1 = dt.copy(deep=True)
for i in output_d.keys():
    dt1[i] = pd.Series(output_d[i])
dt1.head()

,docname,topic,proportion,0,1,2,3,4,5,6,...,90,91,92,93,94,95,96,97,98,99
0,0,"[80, 4, 53, 7, 16, 15, 1, 8, 29, 38]","[0.243127, 0.20963400000000001, 0.108238000000...",0.0,-0.045352,0.0,0.0,-0.209236,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,"[4, 33, 18, 36, 19]","[0.666429, 0.218793, 0.064818, 0.0294089999999...",0.0,0.000000,0.0,0.0,-0.610849,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,[55],[1.0],0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,[72],[1.0],0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,"[0, 1]","[0.553972, 0.446028]",0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
#drop superfluous topic, proportion columns
dt2 = dt1.drop('topic', axis=1)
dt3 = dt2.drop('proportion', axis=1)
dt3.head()

,docname,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,0.0,-0.045352,0.0,0.0,-0.209236,0.0,0.0,-0.099317,-0.039465,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.000000,0.0,0.0,-0.610849,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
from datetime import datetime, date

In [93]:
datetime.strptime(dates[1][:19], '%Y-%m-%d %H:%M:%S')

datetime.datetime(2020, 7, 16, 1, 24, 10)

In [94]:
#convert everything to dates
date_l = []
for date in dates:
    date_l.append(datetime.strptime(date[:19], '%Y-%m-%d %H:%M:%S'))

[datetime.datetime(2020, 7, 16, 1, 24, 9),
 datetime.datetime(2020, 7, 16, 1, 24, 10),
 datetime.datetime(2020, 7, 16, 1, 24, 12),
 datetime.datetime(2020, 7, 16, 1, 24, 21),
 datetime.datetime(2020, 7, 16, 1, 24, 24),
 datetime.datetime(2020, 7, 16, 1, 24, 25),
 datetime.datetime(2020, 7, 16, 1, 24, 26),
 datetime.datetime(2020, 7, 16, 1, 24, 27),
 datetime.datetime(2020, 7, 16, 1, 24, 28),
 datetime.datetime(2020, 7, 16, 1, 24, 29),
 datetime.datetime(2020, 7, 16, 1, 24, 30),
 datetime.datetime(2020, 7, 16, 1, 24, 30),
 datetime.datetime(2020, 7, 16, 1, 24, 31),
 datetime.datetime(2020, 7, 16, 1, 24, 32),
 datetime.datetime(2020, 7, 16, 1, 24, 33),
 datetime.datetime(2020, 7, 16, 1, 24, 34),
 datetime.datetime(2020, 7, 16, 1, 24, 36),
 datetime.datetime(2020, 7, 16, 1, 24, 37),
 datetime.datetime(2020, 7, 16, 1, 24, 37),
 datetime.datetime(2020, 7, 16, 1, 24, 39),
 datetime.datetime(2020, 7, 16, 1, 24, 40),
 datetime.datetime(2020, 7, 16, 1, 24, 42),
 datetime.datetime(2020, 7, 16, 1

In [98]:
dt3['date'] = pd.Series(date_l)
dt3.head()

,docname,0,1,2,3,4,5,6,7,8,...,91,92,93,94,95,96,97,98,99,date
0,0,0.0,-0.045352,0.0,0.0,-0.209236,0.0,0.0,-0.099317,-0.039465,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-07-16 01:24:09
1,1,0.0,0.000000,0.0,0.0,-0.610849,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-07-16 01:24:10
2,2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-07-16 01:24:12
3,3,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-07-16 01:24:21
4,4,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-07-16 01:24:24


In [99]:
dt3.to_csv('./live_sent.csv')